In [86]:
# pip install pandas plotly openpyxl nbformat dash

In [87]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [88]:
fichier_xls = "bnp_export_15_03.xlsx"
df = pd.read_excel(fichier_xls)
df.head()

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Pointage operation,Commentaire operation,Date Extraction
0,77f8493ca24f02dea17cb67965c5258a73fc4fde75e7b2...,2025-03-14,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,NaN,NaN,2025-03-15 10:47:36
1,942d2e6eae9a231cc72985bdc08804aada0b9d36d372a7...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,NaN,NaN,2025-03-15 10:47:36
2,941bb1735f80c2267aac1c36206a2eae90058e2f740d9a...,2025-03-14,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,NaN,NaN,2025-03-15 10:47:36
3,a09fdabc5afb7f14f4275c530543500f1e1294e929fd2f...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,NaN,NaN,2025-03-15 10:47:36
4,4b45642f04eeb3b3f50599b08b28c403fe6cf684a80ae5...,2025-03-13,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,NaN,NaN,2025-03-15 10:47:36


In [89]:
df["Mois"] = df["Date operation"].dt.to_period("M")
df["Type"] = df["Montant operation"].apply(lambda x: "Recette" if x > 0 else "Dépense")

In [90]:
df.dtypes

id                                  object
Date operation              datetime64[ns]
Categorie operation                 object
Sous Categorie operation            object
Libelle operation                   object
Montant operation                  float64
Pointage operation                 float64
Commentaire operation              float64
Date Extraction             datetime64[ns]
Mois                             period[M]
Type                                object
dtype: object

In [91]:
# Filtrer uniquement les dépenses (montant négatif)
data = df
data = data[data["Montant operation"] < 0]
data = data[data["Sous Categorie operation"] != 'Virement interne']
data = data[data["Categorie operation"] != 'Revenus']
df_expenses = data
df_expenses["Montant operation"] = df_expenses["Montant operation"].abs()  # Valeur absolue

In [92]:


# Regrouper par catégorie et sommer les montants
category_expenses = df_expenses.groupby("Categorie operation")["Montant operation"].sum().reset_index()
# Créer le pie chart
fig1 = px.pie(
    category_expenses,
    names="Categorie operation",
    values="Montant operation",
    title="Répartition des dépenses par catégorie",
    hole=0.4,
    color_discrete_sequence=px.colors.qualitative.Set3 
)

fig1.show()

In [93]:
# Agréger les dépenses par jour
daily_expenses = df_expenses.groupby("Date operation")["Montant operation"].sum().reset_index()

# Créer le graphique en ligne
fig2 = px.line(
    daily_expenses,
    x="Date operation",
    y="Montant operation",
    title="Évolution des dépenses dans le temps",
    markers=True,  # Ajoute des points pour chaque jour
    line_shape="spline",  # Lissage des lignes
    color_discrete_sequence=["red"]
)

fig2.update_layout(
    xaxis_title="Date",
    yaxis_title="Montant (€)",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

fig2.show()
